In [4]:
# import api_keys
import requests
import time
import shutil # to save it locally
import json
import os, sys
import re
from os import environ
from client_id import client_id as clientID, client_secret as secret
from datetime import datetime
from flask import Flask, render_template, request, redirect
from PIL import Image, ImageDraw, ImageFont
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from multiprocessing import Process
from pystray import MenuItem as item
import pystray

# Start Application 
# Start Selenium Browser
# request address for user sign-in
# Login to Spotify
# Redirect to Local URL
# Get User's Features Selection
# Get User's tokens and Permissions
# Request for the users current playing song
# Grab the current playing songs uri
# Parse the uri for playist or song code
# Request Image of Code
# Create Directory for Image
# Store Image of Code
# Update Image on new song
# Kill Program and remove memory leaks

In [29]:
# Create tray icon for exit
# Login to Spotify
# Get Users tokens and Permissions
# Request for the users current playing song
# scope:user-read-currently-playing
# get an image
def action():
    pass
def create_image():
    # Generate an image and draw a pattern
#     image = Image.new('RGB', (width, height), color1)
    # make a blank image for the text, initialized to transparent text color
    txt = Image.new("RGBA", size= (50,50), (255,255,255,0))

    # get a font
    fnt = ImageFont.truetype("arial.ttf", 20)
    # get a drawing context
    SCO = ImageDraw.Draw(txt)

    # draw text, half opacity
    SCO.text((10,10), "ScO", font=fnt, fill=(255,255,255,128))

    return SCO
# Create permission prompt and feature choices for the image
def get_permission(browser):
    url = 'https://accounts.spotify.com/authorize'
    header = {
        'client_id': clientID,
        'response_type': 'code',
        'redirect_uri': 'http://localhost:5500/login',
        'state': 't1i2m3e4g5n6o7m8e',
        'scope': 'user-read-currently-playing',
        'show_dialog': 'false'}
    req = requests.get(url,header)
    browser.visit(req.url)

auth_info= ''
@app.route('/login',  methods = ['GET', 'POST'])
def get_login(browser):
    form = SubmissionForm(request.form)
    
    if request.method ='POST' and form.validate():
        urlcode = browser.url
        auth_info = re.search(r'code=(.+)&state=.+',urlcode)[0]
        
        return render_template(
        'login.html',
        year = datetime.now().year
        )
    else:
        return render_template(
            'login.html',
            year = datetime.now().year
        )
    
@app.route('/features', methods = ['GET', 'POST'])
def features():
    if request.method ='POST' and form.validate():
        style ={
            'authcode': authcode,
            'background': form.background.data,
            'size': form.size.data,
            'color' form.color.data:
        }
        return render_template(
            'completed.html',
            year = datetime.now().year
        )
    else:
        return render_template(
            'features.html',
            form = form,
            year = datetime.now().year
        )
    
def get_spotify_song_info(style):
    request_body = json.dumps({
        "market" : 'US',
        "additional_types": 'track'
    })
    query = f"https://api.spotify.com/v1/me/player/currently-playing"
    response = requests.post(
    query,
    data = request_body,
    headers= {
        "Content-Type":"application/json",
        "Authorization":f"Bearer {style['authcode']}"
        }
    )
    if response.status_code == 200:
        response_json= response.json()
        # Grab the current playing songs uri
        try:
            song {'uri':response_json['item']['uri'],
            'name':response_json['item']['name'],
            'background':style['background'],
            'size':style['size'],
            'color':style['color']}
        except:
            song {'uri':response_json['item'][0]['uri'],
            'name':response_json['item']['name'],
            'background':style['background'],
            'size':style['size'],
            'color':style['color']}
    return song
    # Parse the uri for playist or song code if needed or if null
def get_spotify_code_image(song):
    url = f"https://scannables.scdn.co/uri/plain\
          /jpeg/{song['color']}/{song['background']}/{song['size']}/{song['uri']}"
    response = requests.get(url, stream=True)
    # # print(response.status_code)
    # test = Image.open(BytesIO(response))
    if response.status_code == 200:
        # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
        response.raw.decode_content = True

        # Open a local file with wb ( write binary ) permission.
        with open('spotify_code.jpeg','wb') as f:
            shutil.copyfileobj(response.raw, f)

        print('Image sucessfully Downloaded: ','spotify_code.jpeg')
    else:
        print('Image Couldn\'t be retreived')

if __name__ == "__main__":
#     Create system tray icon to be able to terminate program without
#     using the kill cmd or task manager
    image = create_image()
    menu = (item('Info', action), item('Exit', lambda : sys.exit()))
    icon = pystray.Icon("ScO", image, "ScO Menu", menu)
    icon.run()
#     Setup server thread for the local flask app
    HOST = environ.get('SERVER_HOST', 'localhost')
    try:
        PORT = int(environ.get('SERVER_PORT', '5500'))
    except ValueError:
        PORT = 5500
    server = Process(target=app.run(HOST, PORT))
    server.start()
    
#     Start browser for the request redirects and url displayed
    executable_path = {"executable_path": ChromeDriverManager().install()}
    brws = Browser('chrome', **executable_path, headless=False)
    
#     Get the permissions from the user sending the requests and redirect
#     to be shown on the browser
    get_permission(brws)
#     Retrieve the authentication and refresh tokens
    OAuth_tokens = get_tokens(auth_info)
#     Get the features the user would like to have on their image
    features = get_features(brws)
#     remove resources
    brws.close()
#     terminate server once done with the user requests
    server.terminate()
    server.join()
#     Send request with auth info for the song request of the 
#     currently playing song if available
    song_info = get_spotify_song_info(features)
#     Get image of spotify code of the current song
    get_spotify_code_image(song_info)
#     start timer for the system
    start_time=time.perf_counter()
#     spotify_code = CreateSpotifyCode()
#         spotify_code.get_spotify_song_info(song_info)
#  loop through until system terminated
    while(true):
        if (start_time - time.perf_counter)%30 = 0:
            get_spotify_code_image(song_info)
        if (start_time - time.perf_counter)%3600 = 0:
            OAth_tokens = get_tokens(auth_info)
        

In [ ]:
#Status Code 	Description
# 200 	OK - The request has succeeded. The client can read the result of the request in the body and the headers of the response.
# 201 	Created - The request has been fulfilled and resulted in a new resource being created.
# 202 	Accepted - The request has been accepted for processing, but the processing has not been completed.
# 204 	No Content - The request has succeeded but returns no message body.
# 304 	Not Modified. See Conditional requests.
# 400 	Bad Request - The request could not be understood by the server due to malformed syntax. The message body will contain more information; see Response Schema.
# 401 	Unauthorized - The request requires user authentication or, if the request included authorization credentials, authorization has been refused for those credentials.
# 403 	Forbidden - The server understood the request, but is refusing to fulfill it.
# 404 	Not Found - The requested resource could not be found. This error can be due to a temporary or permanent condition.
# 429 	Too Many Requests - Rate limiting has been applied.
# 500 	Internal Server Error. You should never receive this error because our clever coders catch them all … but if you are unlucky enough to get one, please report it to us through a comment at the bottom of this page.
# 502 	Bad Gateway - The server was acting as a gateway or proxy and received an invalid response from the upstream server.
# 503 	Service Unavailable - The server is currently unable to handle the request due to a temporary condition which will be alleviated after some delay. You can choose to resend the request again.